In [ ]:
# continuous batching—相关代码
# 在llm.py的_run_engine()函数中，会循环检测是否还存在未完成的requests，如果存在，则生成单次的推理结果。

def _run_engine(self, use_tqdm: bool) -> List[RequestOutput]:  # 定义_run_engine函数，输入参数use_tqdm决定是否使用进度条
    # Initialize tqdm.  # 初始化tqdm进度条
    if use_tqdm:  # 如果使用tqdm
        num_requests = self.llm_engine.get_num_unfinished_requests()  # 获取未完成请求的数量
        pbar = tqdm(total=num_requests, desc="Processed prompts")  # 创建tqdm进度条对象，设置总数和描述
    # Run the engine.  # 运行引擎
    outputs: List[RequestOutput] = []  # 初始化输出列表，存储每个请求的输出
    while self.llm_engine.has_unfinished_requests():  # 当引擎中还有未完成的请求
        step_outputs = self.llm_engine.step()  # 进行一步处理，获取本次步骤的输出
        for output in step_outputs:  # 遍历本次步骤的每个输出
            if output.finished:  # 如果输出已完成
                outputs.append(output)  # 将输出添加到输出列表中
                if use_tqdm:  # 如果使用tqdm
                    pbar.update(1)  # 更新进度条
    if use_tqdm:  # 如果使用tqdm
        pbar.close()  # 关闭进度条
    # Sort the outputs by request ID.  # 根据请求ID对输出进行排序
    # This is necessary because some requests may be finished earlier than its previous requests.  # 这是必要的，因为某些请求可能比前面的请求更早完成
    outputs = sorted(outputs, key=lambda x: int(x.request_id))  # 按请求ID排序输出
    return outputs  # 返回排序后的输出


In [ ]:
# 在self.llm_engine.step()中，会获取_schedule中需要推理的seqs的原始数据列表。
def step(self) -> List[RequestOutput]:  # 定义step函数，返回RequestOutput的列表
    """Performs one decoding iteration and returns newly generated results.  # 执行一次解码迭代并返回新生成的结果

    This function performs one decoding iteration of the engine. It first  # 此函数执行引擎的一次解码迭代
    schedules the sequences to be executed in the next iteration and the  # 首先调度将在下一次迭代中执行的序列
    token blocks to be swapped in/out/copy. Then, it executes the model  # 以及将要交换的token块（包括调入、调出和复制）
    and updates the scheduler with the model outputs. Finally, it decodes  # 然后执行模型，并使用模型的输出更新调度器
    the sequences and returns the newly generated results.  # 最后对序列进行解码并返回新生成的结果
    """
    seq_group_metadata_list, scheduler_outputs, ignored = self._schedule()  # 调用_schedule函数，获取序列组元数据列表、调度器输出和被忽略的输出
    if scheduler_outputs.is_empty():  # 如果调度器输出为空
        return ignored  # 返回被忽略的输出

    # Execute the model.  # 执行模型
    output = self._run_workers(  # 调用_run_workers函数执行模型
        "execute_model",  # 执行模型的任务名称
        seq_group_metadata_list=seq_group_metadata_list,  # 传入序列组元数据列表
        blocks_to_swap_in=scheduler_outputs.blocks_to_swap_in,  # 传入要调入的块
        blocks_to_swap_out=scheduler_outputs.blocks_to_swap_out,  # 传入要调出的块
        blocks_to_copy=scheduler_outputs.blocks_to_copy,  # 传入要复制的块
    )
    
    return self._process_model_outputs(output, scheduler_outputs) + ignored  # 处理模型输出并返回调度器输出和被忽略的输出的合并结果
